<a href="https://colab.research.google.com/github/mykolesiko/eeg_investigation/blob/diplom/MADE_PCC_leak_beta1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [ ]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [ ]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 256
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="/content/drive/MyDrive/MADE/Project/RACNN_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob
import pickle
from collections import Counter
import scipy
from  scipy import signal
from scipy.fft import fft, fftfreq

In [ ]:
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []
data_dir = './data_preprocessed_python'
files = glob.glob(os.path.join(data_dir, "*.dat"))
data_raw = []
for file_data in files:
    raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
    data.append(raw_data['data'])
    labels.append(raw_data['labels'])
    

In [ ]:
labels_bin = []
for sub in range(32):
#for sub in range(1):
  temp = labels[sub] >= 4.5
  labels_bin.append(temp)
  #print(sum(labels_bin[sub][:, type_emotion]), end=' ')

In [ ]:
labels_bin = np.array(labels_bin)

In [ ]:
LEN_RECORD_IN_SECONDS = 60
NVIDEOS = 40
HCANALS = 9
WCANALS = 9
NTIMES_IN_SAMPLE = 128
NTIMES_IN_SEC = 128
NCANALS = 32
NFEATURES = 32
NSUBJECTS = 32
electrode_matrix = {}
electrode_matrix['FP1'] = [0, 3]
electrode_matrix['FP2'] = [0, 5]
electrode_matrix['AF3'] = [1, 3]
electrode_matrix['AF4'] = [1, 5]
electrode_matrix['F7']  = [2, 0]
electrode_matrix['F3']  = [2, 2]
electrode_matrix['FZ']  = [2, 4]
electrode_matrix['F4']  = [2, 6]
electrode_matrix['F8']  = [2, 8]
electrode_matrix['FC5']  = [3, 1]
electrode_matrix['FC1']  = [3, 3]
electrode_matrix['FC2']  = [3, 5]
electrode_matrix['FC6']  = [3, 7]
electrode_matrix['T7']  = [4, 0]
electrode_matrix['C3']  = [4, 2]
electrode_matrix['CZ']  = [4, 4]
electrode_matrix['C4']  = [4, 6]
electrode_matrix['T8']  = [4, 8]
electrode_matrix['CP5']  = [5, 1]
electrode_matrix['CP1']  = [5, 3]
electrode_matrix['CP2']  = [5, 5]
electrode_matrix['CP6']  = [5, 7]
electrode_matrix['P7']  = [6, 0]
electrode_matrix['P3']  = [6, 2]
electrode_matrix['PZ']  = [6, 4]
electrode_matrix['P4']  = [6, 6]
electrode_matrix['P8']  = [6, 8]
electrode_matrix['PO3'] = [7, 3]
electrode_matrix['PO4'] = [7, 5]
electrode_matrix['O1'] = [8, 3]
electrode_matrix['OZ'] = [8, 4]
electrode_matrix['O2'] = [8, 5]

list_electrodes = ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3',	'T7',	'CP5',	'CP1',	'P3',	'P7',	'PO3',	'O1',	'OZ',	'PZ',	'FP2',	'AF4', 'FZ', 'F4', 'F8', 'FC6',	'FC2',	'CZ', 'C4', 'T8', 'CP6',	'CP2',	'P4', 	'P8',	'PO4',	'O2']
data_dir = './data_preprocessed_python'
TRAIN_SIZE = 0.9
THRESHOLD = 4.5

In [ ]:
pip install mne

In [ ]:
#  from scipy import signal
#  from mne.filter import filter_data

#  data_samples = []
#  labels = []
#  interval = 3
#  shift = 1
#  for sub in range(len(data)): #sub   - человек
#       print(sub)
#       for nvideo in range(NVIDEOS):
#             print(nvideo)
#             temp = data[sub][nvideo, :32, 3 * 128 :]
#             data_sample_all = np.zeros((10, 32, 8064 - 3 * 128), dtype=np.float16)
                    
#             for  s in range(10):
#                 freq_ranges_gc = np.array([2, 3, 4, 7, 8, 9.5, 10.5, 12, 8, 12, 13, 16, 17,20, 21, 29, 13, 29, 30, 50])
#                 freq_beg = freq_ranges_gc[2 * s]
#                 freq_end = freq_ranges_gc[2 * s + 1]
#                 data_sample_all[s, :] = filter_data(temp, 128, freq_beg, freq_end, filter_length = 'auto', verbose = False)#signal.filtfilt(b, a, temp[i])  # данные - это сигнал, который нужно отфильтровать
                
#             for nsec in range(3 * 128, (LEN_RECORD_IN_SECONDS - interval) * 128, 64):
#                    #temp = data[sub][nvideo, :32,  (3 + nsec) * NTIMES_IN_SEC : (3 + nsec + interval) * NTIMES_IN_SEC].copy()
                   
#                    #freq_ranges_gc = np.array([4, 7, 8, 9.5, 10.5, 12, 8, 12, 13, 16, 17,20, 21, 29, 13, 29, 30, 50])
#                    sample = data_sample_all[:, :, nsec : nsec + 128 * 3]
#                    canal_data = sample.copy()
#                    #list_canals = np.arange(NCANALS)
#                    #random.shuffle(list_canals)
#                    #anal_data_new = [canal_data[:, list_canals[i], :] for i in range(32)]
#                    data_sample = []
#                    for s in range(10):
#                         pcc_matrix =  np.corrcoef(canal_data[s]).astype(np.float16)
#                         data_sample.append(pcc_matrix)
#                         #print(pcc_matrix)  
#                    data_sample = np.array(data_sample)            
#                    data_samples.append(data_sample)
#                    labels.append(labels_bin[sub][nvideo, :])


In [ ]:
#del data

In [ ]:
#data_samples = np.array(data_samples)

In [ ]:
#print(data_samples.size)

In [ ]:
#labels = np.array(labels)

In [ ]:
class EmotionDataset(Dataset):
    def __init__ (self, data_samples, labels, inds):  #indexes - индексы видео которые вошли в датасет, data - данные labels - метки бинарные
       super().__init__()
       self.data_samples = data_samples[inds]
       self.labels = labels[inds]
       self.inds = inds
     

    def __len__(self):
       result =  len(self.inds)
       return result

    def __getitem__(self, item):
       sample = {}   
       sample = {"labels": self.labels[item],
                 "data": self.data_samples[item]
       }
       
       if transforms is not None:
           for t in transforms:
                sample = t(sample)
       #print(sample)         
       
       return sample

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

delta (0-3 Hz), theta (4-7
Hz), low alpha (8-9.5 Hz), high alpha (10.5-12 Hz), alpha (8-
12 Hz), low beta (13-16 Hz), mid beta (17-20 Hz), high beta
(21-29 Hz), beta (13-29 Hz), and gamma (30-50 Hz).

In [ ]:

import matplotlib 
from matplotlib import pyplot as plt




class ToTensor(object):

    def __init__(self, ):
        super(ToTensor, self).__init__()

    def __call__(self, sample: dict):
        return {"labels": torch.tensor(sample["labels"], dtype=torch.long),
                "data": torch.tensor(sample["data"], dtype=torch.float32),
                } 
                      
transforms = [ToTensor()]                       

In [ ]:
def get_conv_module(n_input_channels,n_output_channels):
  conv = nn.Conv2d(n_input_channels, n_output_channels, kernel_size = 3, stride=1, padding='same')
  maxpool = nn.MaxPool2d(kernel_size = 2)
  bn = nn.BatchNorm2d(n_output_channels)
  relu = nn.ReLU()
  result = torch.nn.Sequential(conv, maxpool, bn, relu)
  return(result)

In [ ]:
writer = None

In [ ]:
class EmotionNet(torch.nn.Module): 
   def __init__(self, hcanals, wcanals, tures, ntimes_in_sample):
      super().__init__()
      n_input_channels = [10, 32, 64, 128, 256]
      n_output_channels = [32, 64, 128, 256, 512]
      self.convs = nn.ModuleList([get_conv_module(n_input_channels[i], n_output_channels[i]) for i in range(4)])
      self.conv = nn.Conv2d(n_input_channels[4], n_output_channels[4], kernel_size = 4, stride=1, padding='same')
      self.flat = nn.Flatten(1, 3)
      self.fc = nn.Linear(2048, 2)

      
   def forward(self, input):
      #print(f"input_shape = {input.shape}")
      #input = input.unsqueeze(3)
      #input = input.permute(0, 3, 1, 2)
      #input (bs, h=32, w=32, 1)

      output = input
      for i, conv in enumerate(self.convs):
            output = conv(output)
            #print(outputs[i])
      output = self.conv(output)
      output = self.flat(output)
      #output (bs, s * 256,  h=9, w=9)
      output = self.fc(output)
      return output



In [ ]:
def train(model, loader, criterion, optimizer, device, val_dataloader, val_f1_min,  description, type_emotion, batch = None, writer = None):
    model.train()
    train_loss = []
    inputs = []
    #torch.autograd.set_detect_anomaly(True)
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    #for i , batch in enumerate(tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True)):
    for i, batch in enumerate(loader):
            model.train()
            optimizer.zero_grad()
            src  = batch['data'].to(device)
            trg = batch['labels'][:, type_emotion]
            #print(batch)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            loss = criterion(levels_pred, trg) 
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            if writer:
                writer.add_scalar(f'{description}/training loss per batch',
                                  loss.item(),
                                  i)

            # if (i % 100 == 0):
            #     acc, f1 = (calculate_predictions(model, val_dataloader))
            #     if (f1 > val_f1_min):
            #           val_f1_min      = f1
            #           torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join(args.output_dir, f"val.tgz"))
            #break
    return np.mean(train_loss), val_f1_min

def evaluate(model, loader, criterion, device, writer, description, type_emotion):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        for s, batch in enumerate(loader):#, total=len(loader), desc="validating...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]



            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()
            loss = criterion(levels_pred, trg) 
            epoch_loss += loss.item() 


            if writer:
                writer.add_scalar(f'{description}/val loss per batch',
                                  loss.item(),
                                  s)
        
    return epoch_loss / s
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

def calculate_predictions(model, loader, type_emotion, show):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(loader):#, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]
         

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            
            real.extend(trg)
            pred.extend(trg_pred) 

        if show:    
            print(accuracy_score(real, pred)) 
            print(confusion_matrix(real, pred))  
            print(classification_report(real, pred))  
        f1 = ((f1_score(real, pred, average = 'binary', pos_label = 0))  + (f1_score(real, pred, average = 'binary', pos_label = 1)))/2
        return (accuracy_score(real, pred)) , f1
        #plt.hist(real)
def train_loop(description, type_emotion, n_epochs = 10):
    #args.epochs = 10
    #criterion =  fnn.mse_loss
    train_loss_min = 10000
    val_f1_min = -10000

    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    #batch = next(iter(train_dataloader))
    for epoch in range(n_epochs):
          #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
    
          
          train_loss, val_f1_min  = train(model, train_dataloader, criterion, optimizer ,device, val_dataloader, val_f1_min ,  description , type_emotion, None,  writer)
          #if epoch % 500 == 0:
          if writer:
                writer.add_scalar(f"{description}/training loss per epoch",
                                        train_loss,
                                        epoch)
          #print(train_loss)

          if (train_loss < train_loss_min):
                 train_loss_min      = train_loss
                 torch.save({
                         'model_state_dict': model.state_dict(),
                         'optimizer_state_dict': optimizer.state_dict(),
                       },
                       os.path.join(args.output_dir, "train.tgz")
            )  

          val_loss = evaluate(model, val_dataloader, criterion, device,  writer, description , type_emotion)
          # #break
          print(f"train_loss {train_loss:0.5f}  val_loss {val_loss:0.5f}")
          lr_scheduler.step(val_loss)
          if writer:
                writer.add_scalar(f"{description}/val loss per epoch",
                                  val_loss,
                                  epoch)
          acc, f1 = (calculate_predictions(model, val_dataloader, type_emotion, False))
          print(f"acc {acc:0.5f}, f1  {f1:0.5f}")
          if (acc > val_f1_min):
                       val_f1_min      = acc
                       torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join(args.output_dir, f"val_{description}.tgz")) 

In [ ]:
def get_model():
  model = EmotionNet(HCANALS, WCANALS, NFEATURES, NTIMES_IN_SAMPLE).to(device)
  return model
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)


#LEAK

##Valence

In [ ]:
type_emotion  = 0

In [ ]:
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score, accuracy_score
interval = 3
shift = 1
koeff = 0.8
import pandas as pd    
type_emotion  = 0
# #inds = np.arange(2280)
#inds = np.arange(72960)
args.batch_size = 128

# #inds = np.arange(12800)
# random.shuffle(inds)
acc_all = []
f1_all = []
inds = np.arange(len(data_samples))
#inds =  np.arange(72960)
kf = KFold(n_splits=5, shuffle = True)
for fold,  (inds_train, inds_test) in enumerate(kf.split(inds)):
    print(f"fold = {fold}")
         
    #transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
    #transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
    #transforms = [to_pcc_matrix(),   ToTensor()]       
    transforms = [ToTensor()]       
    #transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
    #transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   

    train_dataset = EmotionDataset(data_samples, labels, inds_train)
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=4, 
                                  pin_memory=True, shuffle=True, drop_last=True)

    val_dataset = EmotionDataset(data_samples, labels, inds_test)
    val_dataloader = DataLoader(val_dataset, 4096, num_workers=4,
                              pin_memory=True, shuffle=False, drop_last=False)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = get_model()
    model.apply(initialize_weights)
    criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
    #optimizer = optim.Adam(model.parameters(), lr=3e-4,  weight_decay=0.001)
    optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
    description = f'psd_model_{fold}'
    
    train_loop(description, 0, 30)
    
    model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
    model.load_state_dict(model_state['model_state_dict'])
    acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, True)
    print(f"************f1 = {f1} acc = {acc}********************")
    acc_all.append(acc)
    f1_all.append(f1)
        
    pd.DataFrame(f1_all).to_csv("f1_val_pсс_leak1.csv")
    pd.DataFrame(acc_all).to_csv("acc_val_pcc_leak1.csv")






  

fold = 0


training...:   0%|          | 0/864 [00:00<?, ?it/s]

train_loss 0.59572  val_loss 0.56879
acc 0.75633, f1  0.72771


training...:   0%|          | 0/864 [00:00<?, ?it/s]

train_loss 0.43437  val_loss 0.47245
acc 0.81196, f1  0.79382


...

train_loss 0.00184  val_loss 0.41137
acc 0.95309, f1  0.94938


training...:   0%|          | 0/864 [00:00<?, ?it/s]

train_loss 0.00158  val_loss 0.41155
acc 0.95338, f1  0.94969


training...:   0%|          | 0/864 [00:00<?, ?it/s]

train_loss 0.00157  val_loss 0.40898
acc 0.95341, f1  0.94976
0.9534505208333334
[[ 9470   651]
 [  636 16891]]
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     10121
           1       0.96      0.96      0.96     17527

    accuracy                           0.95     27648
   macro avg       0.95      0.95      0.95     27648
weighted avg       0.95      0.95      0.95     27648

************f1 = 0.9498365484229966 acc = 0.9534505208333334********************


#NO LEAK

## Valence

In [ ]:
type_emotion = 0

In [ ]:
 from scipy import signal
 from mne.filter import filter_data
 from sklearn.model_selection import StratifiedKFold
 from sklearn.metrics import f1_score, accuracy_score


import pandas as pd
interval = 3
shift = 1
k = 5
inds_train = []
inds_test = []
for sub in range(32):  
    inds_train.append([])
    inds_test.append([])
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    acc_sub = []
    f1_sub = []
    for ind_train, ind_test in  balanced_split:
        #print(ind_train, ind_test)
        #количестов меток 1 в трейне и тесте
        #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        inds_train[-1].append(ind_train)
        inds_test[-1].append(ind_test)

 data_samples_train = []
 data_samples_test = []
 labels_train = []
 labels_test = []

 interval = 3
 shift = 1
 for sub in range(len(data)): #sub   - человек
      print(sub)
      for nvideo in range(NVIDEOS):
            print(nvideo)
            temp = data[sub][nvideo, :32, 3 * 128 :]
            data_sample_all = np.zeros((10, 32, 8064 - 3 * 128), dtype=np.float16)
                    
            for  s in range(10):
                freq_ranges_gc = np.array([2, 3, 4, 7, 8, 9.5, 10.5, 12, 8, 12, 13, 16, 17,20, 21, 29, 13, 29, 30, 50])
                freq_beg = freq_ranges_gc[2 * s]
                freq_end = freq_ranges_gc[2 * s + 1]
                data_sample_all[s, :] = filter_data(temp, 128, freq_beg, freq_end, filter_length = 'auto', verbose = False)#signal.filtfilt(b, a, temp[i])  # данные - это сигнал, который нужно отфильтровать
                
            for nsec in range(3 * 128, (LEN_RECORD_IN_SECONDS - interval) * 128, 64):
                   #temp = data[sub][nvideo, :32,  (3 + nsec) * NTIMES_IN_SEC : (3 + nsec + interval) * NTIMES_IN_SEC].copy()
                   
                   #freq_ranges_gc = np.array([4, 7, 8, 9.5, 10.5, 12, 8, 12, 13, 16, 17,20, 21, 29, 13, 29, 30, 50])
                   sample = data_sample_all[:, :, nsec : nsec + 128 * 3]
                   canal_data = sample.copy()
                   #list_canals = np.arange(NCANALS)
                   #random.shuffle(list_canals)
                   #anal_data_new = [canal_data[:, list_canals[i], :] for i in range(32)]
                   data_sample = []
                   for s in range(10):
                        pcc_matrix =  np.corrcoef(canal_data[s]).astype(np.float16)
                        data_sample.append(pcc_matrix)
                        
                        #print(pcc_matrix)  
                   data_sample = np.array(data_sample)            
                   if nvideo in inds_train[sub][0]:
                            data_samples_train.append(data_sample)
                            labels_train.append(labels_bin[sub][nvideo, :])
                   if nvideo in inds_test[sub][0]:
                            data_samples_test.append(data_sample)    
                            labels_test.append(labels_bin[sub][nvideo, :])
                   #data_samples.append(data_sample)
                   #labels.append(labels_bin[sub][nvideo, :])


0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
7
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
8
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
3

In [ ]:
class EmotionDataset_noleak(Dataset):
    def __init__ (self, data_samples, labels):  #indexes - индексы видео которые вошли в датасет, data - данные labels - метки бинарные
       super().__init__()
       self.data_samples = data_samples
       self.labels = labels
       
     

    def __len__(self):
       result =  len(self.labels)
       return result

    def __getitem__(self, item):
       sample = {}   
       sample = {"labels": self.labels[item],
                 "data": self.data_samples[item]
       }
       
       if transforms is not None:
           for t in transforms:
                sample = t(sample)
       #print(sample)         
       
       return sample

In [ ]:
print(inds_train[0][0])

[ 0  2  3  4  6  7  8  9 10 11 12 13 14 15 16 19 20 21 22 23 24 25 26 27
 28 29 31 33 34 35 36 39]


In [ ]:
type_emotion = 0
from sklearn.model_selection import StratifiedKFold


# import pandas as pd
# interval = 3
# shift = 1
# k = 5
# inds_train = []
# inds_test = []
# for sub in range(32):  
#     inds_train.append([])
#     inds_test.append([])
#     X = np.arange(40)
#     y = np.array(labels_bin[sub][:, type_emotion])
#     skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
#     balanced_split = skf.split(X, y)
#     acc_sub = []
#     f1_sub = []
#     for ind_train, ind_test in  balanced_split:
#         #print(ind_train, ind_test)
#         #количестов меток 1 в трейне и тесте
#         #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
#         #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
#         inds_train[-1].append(ind_train)
        #inds_test[-1].append(ind_test)

# inds_train = np.array(inds_train)
# inds_test = np.array(inds_test)
acc_all = []
f1_all = []
for fold in range(0, 1):    
      print(f"fold = {fold}")
         
      #transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
      #transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
      #transforms = [to_pcc_matrix(),   ToTensor()]       
      transforms = [ToTensor()]       
      #transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
      #transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   

      train_dataset = EmotionDataset_noleak(data_samples_train, labels_train)
      train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=2, 
                                    pin_memory=True, shuffle=True, drop_last=True)

      val_dataset = EmotionDataset_noleak(data_samples_test, labels_test)
      val_dataloader = DataLoader(val_dataset, 4096, num_workers=2,
                                pin_memory=True, shuffle=False, drop_last=False)
      
      device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      model = get_model()
      model.apply(initialize_weights)
      criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
      #optimizer = optim.Adam(model.parameters(), lr=3e-4,  weight_decay=0.001)
      optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
      description = f'psd_noleak_model_{fold}'
      
      train_loop(description, 0, 30)
      
      model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
      model.load_state_dict(model_state['model_state_dict'])
      acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, True)
      print(f"************f1 = {f1} acc = {acc}********************")
      acc_all.append(acc)
      f1_all.append(f1)
          
      pd.DataFrame(f1_all).to_csv("f1_val_pсс_noleak.csv")
      pd.DataFrame(acc_all).to_csv("acc_val_pcc_noleak.csv")







fold = 0


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:443: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)


train_loss 0.58454  val_loss 0.80840
acc 0.64902, f1  0.58691
train_loss 0.36969  val_loss 1.03821
acc 0.62634, f1  0.58566
train_loss 0.24280  val_loss 1.27267
acc 0.63487, f1  0.57390
train_loss 0.17238  val_loss 1.63430
acc 0.64804, f1  0.56800
train_loss 0.13714  val_loss 1.74359
acc 0.61104, f1  0.57478
train_loss 0.05865  val_loss 1.92883
acc 0.63462, f1  0.58111
train_loss 0.03803  val_loss 2.14631
acc 0.62952, f1  0.58029
train_loss 0.03007  val_loss 2.35328
acc 0.63191, f1  0.57826
train_loss 0.02433  val_loss 2.56002
acc 0.62731, f1  0.57695
train_loss 0.01815  val_loss 2.58415
acc 0.63260, f1  0.57898
train_loss 0.01720  val_loss 2.61956
acc 0.62970, f1  0.57814
train_loss 0.01633  val_loss 2.64713
acc 0.63129, f1  0.57959
train_loss 0.01570  val_loss 2.70231
acc 0.63267, f1  0.57899
train_loss 0.01493  val_loss 2.69881
acc 0.63158, f1  0.57847
train_loss 0.01466  val_loss 2.71214
acc 0.63162, f1  0.57912
train_loss 0.01464  val_loss 2.71412
acc 0.63064, f1  0.57858
train_lo

In [ ]:
pd.DataFrame(f1_all).to_csv("f1_val_pсс_noleak1.csv")
pd.DataFrame(acc_all).to_csv("acc_val_pcc_noleak1.csv")

In [ ]:
print(f1_all)
print(acc_all)

[0.5869147414906247]
[0.6490162037037037]
